异常预警主要分为两部分:
1. 对已有数据的快速检测,发现异常;
2. 对未来数据进行预测,这里主要是对未来的故障量进行预测;

### 对已有数据进行检测

#### 我们的解决方案

我们的主要思路是$k\sigma$,也就是当数据偏离$k\sigma$时认为数据是异常的.对于正态分布而言,数据分布在$3\sigma$的概率为$97.5\%$.通常对于客服和售后这样的数据可以用泊松分布(Poisson Distribution)来拟合的**\(泊松分布的质量函数和密度函数是不是等价的?)**.除此之外,我们还采用了gamma分布用于拟合我们的数据.
数据首先经过多种变形,如N天移动平均,N天残差,傅立叶变换之后的残差,以及N天增长趋势,然后将这些变形的数据放入上述两种分布之中,通过$N_2$天的数据来得到分布的参数,并判断这$N_2$天的有没有超过$k\sigma$的.


#### Anormaly Detection

业界有许多用于异常检测的算法:

- Holt-winters
- Hot Spot算法




### 对未来数据进行预测

#### 我们的解决方案

这里的未来数据主要是指故障或者客服咨询的数量.
- 首先我们认为售后(客服)数据是带有周期性的,这种周期包括周级别和月级别,通常数据量比较小,一般不会超过两年,因此不会考虑年上面的周期性;
- 其次,不同设备之间的和不同渠道之间的数据会有相互作用相互影响,比如同一时期的设备可能由于生产工艺的问题导致在差不多时期内都会出现成比例的问题,客服渠道的数据也会对售后渠道的数据产生比较大影响;
- 最后,我们认为同一数据前后之间会存在关联,昨天的数据会对今天的数据产生影响,另外故障本身会随着时间的推移而逐步增长的,因此与时间也是存在关系的;

根据上面的分析,我们设计了一个多维预测模型,将上述因素都考虑进去,构成了一个概率图,利用jags(stan)来实现,其中的时间因素主要是通过傅立叶变换实现的$f(x)=\sum_{n=1} ^{\infty}a_n \sin(nx) + b_n\cos(nx) $,我们选取$n=3$来表示weekly周期,$n=7$表示monthy周期.算法的主要实现如下:

1. 时间因素的影响:$g(t)=(k+\alpha (t) ^T \delta)t + (m + \alpha(t)^T \gamma), \gamma _j =  - s_j \delta _j$,其中t是一个由傅立叶变换而成的$vector[k]$,$k$表示时间经过傅立叶展开之后的维度
2. 所有因素的汇总:$f(x)=dpois(wx),x=(g(t),x_{other\,channel},x_{other\,device},x_{current\,device\,former})$


#### Prophet工具
Prophet工具认为y是时间t的函数,将时间t通过傅立叶展开之后,进行函数拟合,类似的方法我们也有采用,将时间t作为其中的一个因素;

#### LSTM方案
我们参考了DeepAR的实现,其网络结构如下:
$(BiLSTM + Dropout)*N + (Gaussian\;Noise) + Dense + RMSE$
整个方案最终表现并不是很好,数据比较容易拟合,在真实的表现中没有达到预期